In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/Shareddrives/NLP Project'

Mounted at /content/drive
/content/drive/Shareddrives/NLP Project


In [ ]:
import json
import os
from eval_script import ClinicalConcept

In [ ]:
# Install libraries to make predictions
!pip install accelerate
!pip install seqeval
!pip install datasets 
!pip install torch
!pip install git+https://github.com/huggingface/transformers

In [ ]:
%cd '/content/drive/Shareddrives/NLP Project/notebooks/subtask 1 NER'
import ner_model
%cd '/content/drive/Shareddrives/NLP Project'

/content/drive/Shareddrives/NLP Project/notebooks/subtask 1 NER
/content/drive/Shareddrives/NLP Project


In [ ]:
def generate_predictions(model_path, input_path, predict_path):
  # Note: train and valid file locations irrelevant;they are just needed for model to run
    arg_string = f'--model_name_or_path {model_path} \
                  --do_predict \
                  --test_file {input_path} \
                  --train_file data/split_data/input/ner_input/ner_input_train.json \
                  --validation_file data/split_data/input/ner_input/ner_input_dev.json \
                  --output_dir {predict_path} \
                  --overwrite_output_dir'

    ner_model.main(arg_string)

In [ ]:
def pre_process_results(predict_path, input_path):
    '''
    predict_path: (str) filepath of predicted labels (in txt file)
    input_path: (str) filepath of input data (in json form)

    return:
        pred_lines: (list) of strings of len = no. sentences
        json_lines: (list) of json-formatted objects, len = no. sentences
    '''
    # predictions
    text = open(f'{predict_path}/predictions.txt', "r")
    pred_lines = [line for line in text]
    # input data
    json_file = open(input_path,'r')
    json_lines = [line for line in json_file]
    assert len(pred_lines)  == len(json_lines)
    return pred_lines, json_lines

In [ ]:
def process_sentences(pred_lines, json_lines):
    '''
    pred_lines: (list) of strings of len = no. sentences
    json_lines: (list) of json-formatted objects, len = no. sentences

    return:
        doc_dic: (dict) of key = doc id (eg "137-04") and values = list of 
                  medication mentions (ClinicalConcept) objects
    '''
    doc_dic = {}
    
    for sent_idx, pred_line in enumerate(pred_lines):
        json_sentence = json.loads(json_lines[sent_idx])
        doc_id = json_sentence['note_id']
        med_list = doc_dic.get(doc_id, [])

        # Process one sentence by iterating over all labels
        for tok_idx, pred_label in enumerate(pred_line.split()):
            if pred_label != "O":
                tok_start, tok_end = json_sentence['token_spans'][tok_idx]
            
                if pred_label == "B-MED":
                    med_number = len(med_list) + 1
                    med_mention = ClinicalConcept(tid = f"T{med_number}",
                                                  start=tok_start,
                                                  end=tok_end,
                                                  text=json_sentence['tokens'][tok_idx],
                                                  ttype="Drug")
                    med_list.append(med_mention)

                elif pred_label == "I-MED":
                    # modify prev med mention to add on the I-MED
                    med_list[-1].end = tok_end
                    med_list[-1].text = med_list[-1].text + " " + json_sentence['tokens'][tok_idx]
        
        # update list for doc with changes
        doc_dic[doc_id] = med_list
    return doc_dic

In [ ]:
def write_to_ann(doc_dic, ann_path):
    # Write results to ann files
    for ann_file, mention_list in doc_dic.items():
        file_name = f'{ann_path}/{ann_file}.ann'
        if os.path.exists(file_name):
            os.remove(file_name)
        with open(file_name, 'w') as ann:
            for mention in mention_list:
                # Note: format between different elements is very specific
                mention_str = f'{mention.rid}\tUndetermined {mention.start} {mention.end}\t{mention.text}\n'
                ann.writelines(mention_str)
                event_str = f'''{mention.rid.replace("T","E")}\tUndetermined:{mention.rid}\n'''
                ann.writelines(event_str)



In [ ]:
# Main program execution
def go(model_path, predict_path, input_path, ann_path):
    '''
    model_path: (str) filepath of NER model
    predict_path: (str) filepath of predicted labels (in txt file)
    input_path: (str) filepath of input data (in json form)
    ann_path: (str) filepath to store new ann files
    '''
    generate_predictions(model_path, input_path, predict_path)
    pred_lines, json_lines = pre_process_results(predict_path, input_path)
    doc_dic = process_sentences(pred_lines, json_lines)
    write_to_ann(doc_dic, ann_path)

# filepaths
model_path = 'data/models/NER'
predict_path = 'data/models/NER_predict'
input_path = 'data/split_data/input/ner_input/ner_input_test.json'
ann_path = 'data/split_data/output/ner_predicted_annotations/'

go(model_path, predict_path, input_path, ann_path)

[INFO|training_args.py:1014] 2022-03-11 18:40:54,511 >> PyTorch: setting up devices
[INFO|training_args.py:877] 2022-03-11 18:40:54,514 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


03/11/2022 18:40:54 - WARNING - ner_model - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/11/2022 18:40:54 - INFO - ner_model - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO|configuration_utils.py:647] 2022-03-11 18:40:55,082 >> loading configuration file data/models/NER/config.json
[INFO|configuration_utils.py:685] 2022-03-11 18:40:55,085 >> Model config BertConfig {
  "_name_or_path": "data/models/NER",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "ner",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-MED",
    "1": "I-MED",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-MED": 0,
    "I-MED": 1,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28

03/11/2022 18:40:57 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-c20d063c61899a88/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-6770ea8db1dbea12.arrow
03/11/2022 18:40:57 - INFO - ner_model - *** Predict ***


[INFO|trainer.py:571] 2022-03-11 18:40:57,395 >> The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, token_spans, tokens, note_id. If ner_tags, token_spans, tokens, note_id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2403] 2022-03-11 18:40:57,407 >> ***** Running Prediction *****
[INFO|trainer.py:2405] 2022-03-11 18:40:57,412 >>   Num examples = 1984
[INFO|trainer.py:2408] 2022-03-11 18:40:57,414 >>   Batch size = 8


03/11/2022 18:41:23 - INFO - datasets.metric - Removing /root/.cache/huggingface/metrics/seqeval/default/default_experiment-1-0.arrow


[INFO|modelcard.py:460] 2022-03-11 18:41:23,596 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}}


***** predict metrics *****
  predict_accuracy           =     0.9982
  predict_f1                 =     0.9451
  predict_loss               =     0.0109
  predict_precision          =     0.9385
  predict_recall             =     0.9518
  predict_runtime            = 0:00:25.74
  predict_samples_per_second =     77.051
  predict_steps_per_second   =      9.631


In [ ]:
# try evaluation script on test data
!python3 notebooks/eval_script.py data/split_data/test/ data/split_data/ner_predicted_annotations/


******************** Evaluation n2c2 2022 Track 1 ********************
************* Contextualized Medication Event Extraction *************

*********************** Medication Extraction ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
                Drug  0.9519  0.9439  0.9479    0.9760  0.9677  0.9718


************************ Event Classification ************************
                      ------- strict -------    ------ lenient -------
                      Prec.   Rec.    F(b=1)    Prec.   Rec.    F(b=1)
         Disposition  0.0000  0.0000  0.0000    0.0000  0.0000  0.0000
       Nodisposition  0.0000  0.0000  0.0000    0.0000  0.0000  0.0000
        Undetermined  0.0849  0.9677  0.1560    0.0849  0.9677  0.1560
                      ------------------------------------------------
     Overall (micro)  0.0849  0.0842  0.0845    0.0849  0.0842  0.0845
  